### Using Azure Storage File Share client library for Python to write UC data to Azure File Share

##### Steps
1. Load data from UC table into Pandas dataframe
2. Stream dataframe into cluster memory as csv string
3. Setup connetion onto Azure File Share (storage account connection string, refer to image in repository)
4. Upload in-memory csv string to file share

##### Source
[Azure File Share Python SDK](https://learn.microsoft.com/en-us/python/api/overview/azure/storage-file-share-readme?view=azure-python)

![](connection-string.png)

In [0]:
import io
from azure.storage.fileshare import ShareFileClient

# in-memory text stream setup
csv_buffer = io.StringIO()

# read data from UC and convert to pandas dataframe
df = spark.table("<catalog>.<schema>.<table>").toPandas()

# load csv string into memory, point string to start 
# Note - if this is a large dataframe you may need to increase job cluster RAM or
# you may need to first chunk the dataframe, write each dataframe to a UC volume, then read 
# each smaller dataframe from the volume bcack to a pandas dataframe and then stream
df.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)

# azure storage account connection string - can copy from Azure portal directly
connection_string="DefaultEndpointsProtocol=https;AccountName=<account_name>;AccountKey=<account_key>;EndpointSuffix=core.windows.net"

# create connection to file share and name of path of file to write
file_client = ShareFileClient.from_connection_string(
    conn_str=connection_string,
    share_name="myshare", # update this name
    file_path="mydir/test.csv" # update this path
)

# stream csv string from memory to file share
file_client.upload_file(csv_buffer.getvalue())